In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
df = pd.read_csv("/kaggle/input/titanic/train.csv")
testdf = pd.read_csv("/kaggle/input/titanic/test.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
#cleaning training dataset

dfs = df.drop('Name', axis = 1)
dfs = dfs.drop ('Ticket', axis =1)
dfs = dfs.drop ('Cabin', axis =1)
dfs['Sex']= dfs['Sex'].apply(lambda x: 1 if x == 'male' else 0)
dfs['Age'] = dfs['Age'].fillna(dfs['Age'].mean())
dfs['Age'].isna().sum()
dfs.iloc[:, 8] = dfs.iloc[:, 8].replace({'Q':'1', 'C':'2', 'S':'3'})
dfs['Embarked'] = pd.to_numeric(dfs['Embarked'], errors='coerce')
dfs['Embarked'] = dfs['Embarked'].fillna(dfs['Embarked'].mean())
dfs.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,3.0
1,2,1,1,0,38.0,1,0,71.2833,2.0
2,3,1,3,0,26.0,0,0,7.9250,3.0
3,4,1,1,0,35.0,1,0,53.1000,3.0
4,5,0,3,1,35.0,0,0,8.0500,3.0


In [4]:
dfs.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Sex              int64
Age            float64
SibSp            int64
Parch            int64
Fare           float64
Embarked       float64
dtype: object

In [5]:
testdfs = testdf.drop('Name', axis = 1)
testdfs = testdfs.drop ('Ticket', axis =1)
testdfs = testdfs.drop ('Cabin', axis =1)
testdfs['Sex']= testdfs['Sex'].apply(lambda x: 1 if x == 'male' else 0)
testdfs['Age'] = testdfs['Age'].fillna(testdfs['Age'].mean())
testdfs['Age'].isna().sum()
testdfs['Fare'] = testdfs['Fare'].fillna(testdfs['Fare'].mean())
testdfs.iloc[:, 7] = testdfs.iloc[:, 7].replace({'Q':'1', 'C':'2', 'S':'3'})
testdfs['Embarked'] = pd.to_numeric(testdfs['Embarked'], errors='coerce')
testdfs.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,7.8292,1
1,893,3,0,47.0,1,0,7.0000,3
2,894,2,1,62.0,0,0,9.6875,1
3,895,3,1,27.0,0,0,8.6625,3
4,896,3,0,22.0,1,1,12.2875,3


In [6]:
testdfs



,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.50000,0,0,7.8292,1
1,893,3,0,47.00000,1,0,7.0000,3
2,894,2,1,62.00000,0,0,9.6875,1
3,895,3,1,27.00000,0,0,8.6625,3
4,896,3,0,22.00000,1,1,12.2875,3
...,...,...,...,...,...,...,...,...
413,1305,3,1,30.27259,0,0,8.0500,3
414,1306,1,0,39.00000,0,0,108.9000,2
415,1307,3,1,38.50000,0,0,7.2500,3
416,1308,3,1,30.27259,0,0,8.0500,3


In [7]:
testdfs.isna().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [8]:
#splitting 

from sklearn.model_selection import train_test_split

x=dfs[['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
y=dfs['Survived']
test=testdfs[['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)

In [10]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_trains = scaler.fit_transform(x_train)
x_tests =scaler.fit_transform(x_test)
xs = scaler.fit_transform(x)

In [11]:
traindf = x_train.join(y_train)
testdf = x_test.join(y_test)

In [12]:
traindf

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
739,740,3,1,29.699118,0,0,7.8958,3.0,0
875,876,3,0,15.000000,0,0,7.2250,2.0,1
824,825,3,1,2.000000,4,1,39.6875,3.0,0
694,695,1,1,60.000000,0,0,26.5500,3.0,0
215,216,1,0,31.000000,1,0,113.2750,2.0,1
...,...,...,...,...,...,...,...,...,...
172,173,3,0,1.000000,1,1,11.1333,3.0,1
866,867,2,0,27.000000,1,0,13.8583,2.0,1
668,669,3,1,43.000000,0,0,8.0500,3.0,0
334,335,1,0,29.699118,1,0,133.6500,3.0,1


In [13]:
#performing logistic regression
'''
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression()
lr.fit(xs,y)
predict=lr.predict(test)'''

'\nfrom sklearn.linear_model import LogisticRegression\nlr=LogisticRegression()\nlr.fit(xs,y)\npredict=lr.predict(test)'

In [14]:
from sklearn.ensemble import RandomForestRegressor

forest = RandomForestRegressor()
forest.fit(x_train, y_train)

RandomForestRegressor()

In [15]:
forest.score(x_test, y_test)

0.5298229491173416

In [16]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=80, max_depth=5, random_state=1)
model.fit(x_train, y_train)

predictions = model.predict(test)

In [17]:
model.score(x_test, y_test)

0.8547486033519553

In [18]:
'''from sklearn.model_selection import GridSearchCV

forest = RandomForestRegressor()

param_grid = (
    "n_estimators": [3, 10, 30],
    "max_features": [2, 4, 6, 8]
)

grid_search = GridSearchCV(forest, param_grid, cv=5,
                          scoring="neg_mean_squared_error",
                          return_train_score=True)

grid_seach.fit(x_train, y_train)'''

'from sklearn.model_selection import GridSearchCV\n\nforest = RandomForestRegressor()\n\nparam_grid = (\n    "n_estimators": [3, 10, 30],\n    "max_features": [2, 4, 6, 8]\n)\n\ngrid_search = GridSearchCV(forest, param_grid, cv=5,\n                          scoring="neg_mean_squared_error",\n                          return_train_score=True)\n\ngrid_seach.fit(x_train, y_train)'

In [19]:
forest.score(x_test, y_test)

0.5298229491173416

In [20]:
#lr.score(x_tests, y_test)

In [21]:
output = pd.DataFrame({'PassengerId': test.PassengerId, 'Survived': predictions})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
